In [11]:
! pip install  implicit


In [12]:
import pandas as pd
import numpy as np
from userknn import UserKnn
from implicit.nearest_neighbours import CosineRecommender
import pickle

In [13]:
interactionset = pd.read_csv("interactions.csv").rename(columns={'last_watch_dt':'datetime'})
interactionset["datetime"] = pd.to_datetime(interactionset["datetime"])

In [14]:
interactionset.head()

,user_id,item_id,datetime,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [15]:
itemcount = interactionset.groupby("item_id")['user_id'].count().reset_index().rename(columns={'user_id':'count'})

In [16]:
top10popular = itemcount.sort_values("count",ascending=False)['item_id'].values[:10]

In [17]:
max_date = interactionset['datetime'].max()

train = interactionset[(interactionset['datetime'] < max_date - pd.Timedelta(days=7))]
test = interactionset[(interactionset['datetime'] >= max_date - pd.Timedelta(days=7))]
warmusers = train['user_id'].unique()

# оставляем только теплых пользователей в тесте
warmtest = test[test['user_id'].isin(warmusers)]
coldtest = test[~test['user_id'].isin(warmusers)]

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (4985269, 5)
test: (490982, 5)


In [18]:
N_USERS = 50
model = CosineRecommender(N_USERS)

userknn = UserKnn(model, N_USERS)
userknn.fit(train)
with open("userKNN.pkl", 'wb') as f:
    pickle.dump(userknn, f)

KeyboardInterrupt: 

In [44]:
def ranksum (rank, top10popular):
    ranknew = top10popular[rank+1:]
    return ranknew


def ranksum_list (rank):
    ranklist = list(range(rank+1,11))
    return ranklist

In [57]:
with open("userKNN.pkl", 'rb') as f:
    userknn = pickle.load(f)
    

/var/folders/st/tcyhwb8n4ssbtcbndbc4_x0r0000gp/T/ipykernel_1501/387486582.py:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  userknn = pickle.load(f)
/var/folders/st/tcyhwb8n4ssbtcbndbc4_x0r0000gp/T/ipykernel_1501/387486582.py:2: DeprecationWarning: Please use `coo_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.coo` namespace is deprecated.
  userknn = pickle.load(f)


In [49]:
userknn.users_mapping

{176549: 0,
 699317: 1,
 656683: 2,
 864613: 3,
 964868: 4,
 1032142: 5,
 1016458: 6,
 884009: 7,
 648682: 8,
 791466: 9,
 988709: 10,
 667487: 11,
 602509: 12,
 5324: 13,
 15607: 14,
 646903: 15,
 616003: 16,
 927973: 17,
 215229: 18,
 896751: 19,
 931816: 20,
 916294: 21,
 916352: 22,
 743269: 23,
 181144: 24,
 841859: 25,
 81786: 26,
 1068939: 27,
 1057871: 28,
 88447: 29,
 141674: 30,
 68478: 31,
 158012: 32,
 127668: 33,
 589939: 34,
 161176: 35,
 21723: 36,
 377423: 37,
 127724: 38,
 985101: 39,
 205710: 40,
 257903: 41,
 193736: 42,
 513902: 43,
 717091: 44,
 163922: 45,
 568405: 46,
 474544: 47,
 311805: 48,
 787115: 49,
 727498: 50,
 35423: 51,
 897955: 52,
 580093: 53,
 957005: 54,
 739920: 55,
 884307: 56,
 77216: 57,
 496279: 58,
 421236: 59,
 290505: 60,
 750995: 61,
 261118: 62,
 806099: 63,
 804607: 64,
 233971: 65,
 785328: 66,
 1072552: 67,
 820464: 68,
 810825: 69,
 934110: 70,
 983574: 71,
 222359: 72,
 94135: 73,
 1022922: 74,
 312347: 75,
 769350: 76,
 108693: 77,


In [58]:
def recomend(usersid:pd.DataFrame,toppopular, Nrecs = 10):
    
    warmusers = list(userknn.users_mapping.keys())
  

    coldtest = usersid[~usersid['user_id'].isin(warmusers)]
    warmtest = usersid[usersid['user_id'].isin(warmusers)]

    #формируем топ популярных для холодных пользователей
    coldtrecomend = pd.DataFrame({'user_id': coldtest["user_id"], 'item_id': [top10popular] * len(coldtest)})
    coldtrecomend = coldtrecomend.explode(column = "item_id")
    coldtrecomend['rank'] = coldtrecomend.groupby('user_id').cumcount() + 1

     #формируем предсказание по горячим пользователям
    warmpredict = userknn.predict(warmtest)[["user_id", 'item_id', 'rank']]
    warmpredictmax  = warmpredict.groupby('user_id')['rank'].max().reset_index()
    users_wo_recomends = warmpredictmax[warmpredictmax['rank'] < 10]

    users_wo_recomends['rank'].apply(lambda x: ranksum(x, top10popular))

    users_wo_recomends["recomendnew"] = users_wo_recomends['rank'].apply(lambda x: ranksum(x, top10popular))

    dataset_recomend_detail = users_wo_recomends[['user_id',"recomendnew"]]\
        .explode(column = "recomendnew").rename(columns = {"recomendnew":"item_id"}).reset_index(drop = True)


    dataset_warm_pred10 = pd.concat([warmpredict[['user_id', 'item_id']],dataset_recomend_detail],ignore_index=True)
    dataset_warm_pred10['rank'] = dataset_warm_pred10.groupby('user_id').cumcount() + 1
    dataset_warm_pred10 = dataset_warm_pred10[dataset_warm_pred10['rank'] <= 10]

    return pd.concat([dataset_warm_pred10,coldtrecomend],ignore_index=True)

In [59]:
recomend(pd.DataFrame({'user_id': test['user_id'].unique()}),top10popular)

/var/folders/st/tcyhwb8n4ssbtcbndbc4_x0r0000gp/T/ipykernel_1501/2372603813.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_wo_recomends["recomendnew"] = users_wo_recomends['rank'].apply(lambda x: ranksum(x, top10popular))


,user_id,item_id,rank
0,1097544,12768,1
1,1097544,4689,2
2,1097544,15942,3
3,1097544,7829,4
4,1097544,4495,5
...,...,...,...
1818389,697262,3734,6
1818390,697262,2657,7
1818391,697262,4880,8
1818392,697262,142,9


In [38]:
top10popular

array([10440, 15297,  9728, 13865,  4151,  3734,  2657,  4880,   142,
        6809])

In [ ]:
warmpredictknn = userknn.predict(warmtest)
#userknn.predict(coldtest)

In [43]:
warmknn = warmpredictknn.groupby('user_id')['rank'].max().reset_index()


users_wo_recomends = warmknn[warmknn['rank'] < 10]

users_wo_recomends['rank'].apply(lambda x: ranksum(x, top10popular))

users_wo_recomends["recomendnew"] = users_wo_recomends['rank'].apply(lambda x: ranksum(x, top10popular))

#users_wo_recomends["ranknew"] = users_wo_recomends["rank"].apply(ranksum_list)

dataset_recomend_detail = users_wo_recomends[['user_id',"recomendnew"]]\
    .explode(column = "recomendnew").rename(columns = {"recomendnew":"item_id"}).reset_index(drop = True)

#dataset_rank_detail = users_wo_recomends[['user_id',"ranknew"]]\
#    .explode(column = "ranknew").rename(columns = {"ranknew":"rank"}).reset_index(drop = True)

#pd.concat([dataset_recomend_detail, dataset_rank_detail], axis = 1,ignore_index=True)
dataset_warm_pred10 = pd.concat([warmpredictknn[['user_id', 'item_id']],dataset_recomend_detail],ignore_index=True)
dataset_warm_pred10['rank'] = dataset_warm_pred10.groupby('user_id').cumcount() + 1
dataset_warm_pred10

/var/folders/st/tcyhwb8n4ssbtcbndbc4_x0r0000gp/T/ipykernel_1501/2573832778.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_wo_recomends["recomendnew"] = users_wo_recomends['rank'].apply(lambda x: ranksum(x, top10popular))


,user_id,item_id,rank
0,1097544,12768,1
1,1097544,4689,2
2,1097544,15942,3
3,1097544,7829,4
4,1097544,4495,5
...,...,...,...
1164509,1097525,2657,6
1164510,1097525,4880,7
1164511,1097525,142,8
1164512,1097525,6809,9
